<a href="https://colab.research.google.com/github/edwardleetenafly/LA-net/blob/main/lanet_week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install nibabel scipy scikit-image tqdm


In [3]:
import nibabel as nib
import numpy as np
from pathlib import Path

root = Path("/content/drive/MyDrive/la_net")

# Pick one case
case_id = "la_003"
img = nib.load(root / "imagesTr" / f"{case_id}.nii")
msk = nib.load(root / "labelsTr" / f"{case_id}.nii")

img_data = img.get_fdata()
msk_data = msk.get_fdata()

print("Image shape:", img_data.shape)
print("Mask shape:", msk_data.shape)
print("Image intensity range:", img_data.min(), img_data.max())

print("Voxel spacing:", img.header.get_zooms())


Image shape: (320, 320, 130)
Mask shape: (320, 320, 130)
Image intensity range: 0.0 1999.0
Voxel spacing: (np.float32(1.25), np.float32(1.25), np.float32(1.37))


In [4]:
import scipy.ndimage as ndimage

def resample_to_isotropic(volume, spacing, new_spacing=(1,1,1), order=1):
    zoom = np.array(spacing) / np.array(new_spacing)
    return ndimage.zoom(volume, zoom, order=order)

img_spacing = img.header.get_zooms()
msk_spacing = msk.header.get_zooms()

img_resampled = resample_to_isotropic(img_data, img_spacing, new_spacing=(1,1,1), order=1)
msk_resampled = resample_to_isotropic(msk_data, msk_spacing, new_spacing=(1,1,1), order=0)

print("Resampled image shape:", img_resampled.shape)
print("Resampled mask shape:", msk_resampled.shape)


Resampled image shape: (400, 400, 178)
Resampled mask shape: (400, 400, 178)


In [5]:
import numpy as np

def normalize(volume):
    mean = np.mean(volume)
    std = np.std(volume)
    return (volume - mean) / (std + 1e-8)

# img_resampled must already exist from Wednesday
img_norm = normalize(img_resampled)

print("Mean after normalization:", img_norm.mean())
print("Std after normalization:", img_norm.std())


Mean after normalization: 1.0037653604679312e-15
Std after normalization: 0.9999999999617678


In [6]:
import numpy as np

def pad_or_crop(volume, target_shape=(128,128,128)):
    out = np.zeros(target_shape, dtype=volume.dtype)

    orig = np.array(volume.shape)
    targ = np.array(target_shape)

    # start index in original
    start = np.maximum((orig - targ)//2, 0)
    # start index in output
    paste_start = np.maximum((targ - orig)//2, 0)

    # slices for original and output
    src = tuple(slice(s, s + min(o, t)) for s, o, t in zip(start, orig, targ))
    dst = tuple(slice(p, p + (ss.stop - ss.start)) for p, ss in zip(paste_start, src))

    out[dst] = volume[src]
    return out

# img_norm (from Thursday) and msk_resampled (from Wednesday) must exist
img_final = pad_or_crop(img_norm)
msk_final = pad_or_crop(msk_resampled)

print("Final image shape:", img_final.shape)
print("Final mask shape:", msk_final.shape)


Final image shape: (128, 128, 128)
Final mask shape: (128, 128, 128)


In [7]:
from pathlib import Path
import numpy as np

prep_dir = Path("/content/drive/MyDrive/la_net/preprocessed")
prep_dir.mkdir(exist_ok=True)

case_id = "la_003"   # change if testing other cases

np.save(prep_dir / f"{case_id}_image.npy", img_final.astype(np.float32))
np.save(prep_dir / f"{case_id}_mask.npy", msk_final.astype(np.uint8))

print("Saved:", f"{case_id}_image.npy and {case_id}_mask.npy")


Saved: la_003_image.npy and la_003_mask.npy


In [9]:
import nibabel as nib
import numpy as np
from pathlib import Path
import scipy.ndimage as ndimage
import pandas as pd
from tqdm import tqdm

root = Path("/content/drive/MyDrive/la_net")
manifest = pd.read_csv(root / "manifest.csv")

prep_dir = root / "preprocessed"
prep_dir.mkdir(exist_ok=True)

def resample_to_isotropic(volume, spacing, new=(1,1,1), order=1):
    zoom = np.array(spacing) / np.array(new)
    return ndimage.zoom(volume, zoom, order=order)

def normalize(v):
    return (v - np.mean(v)) / (np.std(v) + 1e-8)

def pad_or_crop(volume, target=(128,128,128)):
    out = np.zeros(target, dtype=volume.dtype)

    orig = np.array(volume.shape)
    targ = np.array(target)

    start = np.maximum((orig - targ)//2, 0)
    paste_start = np.maximum((targ - orig)//2, 0)

    src = tuple(slice(s, s + min(o, t)) for s, o, t in zip(start, orig, targ))
    dst = tuple(slice(p, p + (ss.stop - ss.start)) for p, ss in zip(paste_start, src))

    out[dst] = volume[src]
    return out

for _, row in tqdm(manifest.iterrows(), total=len(manifest)):
    cid = row["case_id"]

    img = nib.load(row["image_path"])
    msk = nib.load(row["mask_path"])

    img_d = img.get_fdata()
    msk_d = msk.get_fdata()

    img_r = resample_to_isotropic(img_d, img.header.get_zooms(), order=1)
    msk_r = resample_to_isotropic(msk_d, msk.header.get_zooms(), order=0)

    img_n = normalize(img_r)

    img_f = pad_or_crop(img_n)
    msk_f = pad_or_crop(msk_r)

    np.save(prep_dir / f"{cid}_image.npy", img_f.astype(np.float32))
    np.save(prep_dir / f"{cid}_mask.npy", msk_f.astype(np.uint8))

print("Saved all preprocessed data to LA-Net.")


100%|██████████| 20/20 [03:01<00:00,  9.09s/it]

Saved all preprocessed data to LA-Net.
